In [1]:
import pandas as pd
import os
import numpy as np
import datetime
import matplotlib.pyplot as plt
#import geopandas as gpd
import shapely as shp
#import descartes
#from shapely.geometry import Point, Polygon
import matplotlib.dates as mdates
%matplotlib inline
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import branca
import geopandas

import folium # package for making maps, please make sure to use a version older than 1.0.0.
from wordcloud import WordCloud # A package that will allow us to make a wordcloud
from scipy.stats import ttest_ind # A module for Python machine learning--we'll stick to T-Tests here
from IPython.display import display
from folium.plugins import TimeSliderChoropleth
#from time_slider_choropleth import TimeSliderChoropleth
plt.rcParams["figure.figsize"] = (8,5)

In [2]:
url = 'https://www.dropbox.com/s/za6yp2mz6p560o1/uber_trips_2014_nta.csv.gz?dl=1'
uber_trips_2014_nta=pd.read_csv(url, compression='gzip')

In [3]:
uber2015=pd.read_csv("C:/Users/Rodrigo Bernal/Desktop/DS4A_workspace/Datathon/Dataset/Dataset/uber_trips_2015.csv")

In [4]:
url = 'https://www.dropbox.com/s/pjovxyr8w6x6y5x/zones.csv?dl=1'
zones=pd.read_csv(url)

In [5]:
uber_trips_2014_nta = pd.merge(uber_trips_2014_nta,zones, how='left' ,left_on=['pickup_nta'],right_on=['nta_code'])

In [6]:
uber_trips_2015_nta = pd.merge(uber2015,zones, how='left' ,left_on=['pickup_location_id'],right_on=['location_id'])
del uber2015

In [7]:
uber_trips_2014_nta.head()
del uber_trips_2014_nta['Unnamed: 0']
del uber_trips_2014_nta['pickup_latitude']
del uber_trips_2014_nta['pickup_longitude']
del uber_trips_2014_nta['base']
del uber_trips_2014_nta['pickup_point']
del uber_trips_2014_nta['pickup_nta']

In [8]:
del uber_trips_2015_nta['pickup_location_id']
del uber_trips_2015_nta['dispatch_base']
del uber_trips_2015_nta['affiliate_base']
uber_trips_2015_nta.head()
uber_trips=uber_trips_2014_nta.append(uber_trips_2015_nta, ignore_index=True)
del uber_trips_2014_nta
del uber_trips_2015_nta

In [10]:
uber_trips.head()

,pickup_datetime,location_id,borough,zone,service_zone,nta_code
0,4/1/14 0:11,NaN,NaN,NaN,NaN,NaN
1,4/1/14 0:17,NaN,NaN,NaN,NaN,NaN
2,4/1/14 0:21,4.0,Manhattan,Alphabet City,Yellow Zone,MN22
3,4/1/14 0:21,79.0,Manhattan,East Village,Yellow Zone,MN22
4,4/1/14 0:28,161.0,Manhattan,Midtown Center,Yellow Zone,MN17


In [9]:
uber_trips_by_nta=pd.DataFrame(uber_trips.groupby(['nta_code']).size(), columns=['number_of_trips'])
uber_trips_by_nta.reset_index(inplace=True)

In [12]:
uber_trips_by_nta.head()

,nta_code,number_of_trips
0,BK09,152440
1,BK17,15552
2,BK19,5214
3,BK21,6222
4,BK25,19570


In [10]:
import geopandas as gpd
file = gpd.read_file("C:\\Users\\Rodrigo Bernal\\Desktop\\DS4A_workspace\\Datathon\\Dataset\\Dataset\\nynta_19c\\nynta.shp")
file.to_file("nynta.json", driver="GeoJSON")

In [11]:
file.rename(columns={'NTACode': 'nta_code'}, inplace=True)

In [32]:
#folium.GeoJson(file).add_to(heatmap)


In [12]:
min_cn, max_cn = uber_trips_by_nta['number_of_trips'].quantile([0.01,0.99]).apply(round, 2)
colormap = branca.colormap.LinearColormap(
    colors=['white','yellow','orange','red','darkred'],
    #index=beat_cn['count'].quantile([0.2,0.4,0.6,0.8]),b
    vmin=min_cn,
    vmax=max_cn
)

In [13]:
trips_data = file.join(uber_trips_by_nta.set_index("nta_code"), how = "left", on = "nta_code")
trips_data
import numpy
trips_data['number_of_trips'].fillna(0, inplace = True)

In [85]:
trips_data

,BoroCode,BoroName,CountyFIPS,nta_code,NTAName,Shape_Leng,Shape_Area,geometry,number_of_trips
0,3,Brooklyn,047,BK88,Borough Park,39247.228074,5.400502e+07,"POLYGON ((990897.900 169268.121, 990588.252 16...",17452.0
1,4,Queens,081,QN51,Murray Hill,33266.904811,5.248828e+07,"POLYGON ((1038593.459 221913.355, 1039369.281 ...",7115.0
2,4,Queens,081,QN27,East Elmhurst,19816.711538,1.972685e+07,"POLYGON ((1022728.275 217530.808, 1023052.645 ...",15113.0
3,4,Queens,081,QN07,Hollis,20976.335837,2.288777e+07,"POLYGON ((1051539.660 201007.013, 1051920.726 ...",1996.0
4,1,Manhattan,061,MN06,Manhattanville,17040.686548,1.064708e+07,"POLYGON ((999174.261 238490.105, 999084.570 23...",21386.0
...,...,...,...,...,...,...,...,...,...
190,4,Queens,081,QN47,Ft. Totten-Bay Terrace-Clearview,39625.841784,4.629259e+07,"POLYGON ((1048560.005 223080.083, 1048513.939 ...",3775.0
191,5,Staten Island,085,SI05,New Springville-Bloomfield-Travis,109207.917615,3.277686e+08,"POLYGON ((929972.226 170693.326, 930823.112 17...",0.0
192,5,Staten Island,085,SI24,Todt Hill-Emerson Hill-Heartland Village-Light...,75942.003026,1.848815e+08,"POLYGON ((957104.002 161758.179, 957233.148 16...",1861.0
193,1,Manhattan,061,MN11,Central Harlem South,16621.023753,1.443930e+07,"POLYGON ((1000369.738 233326.695, 1000233.777 ...",0.0


In [14]:
trips_data
manhattan_data=trips_data[trips_data['BoroName']=='Manhattan']

In [15]:
from IPython.display import HTML, display
heatmap=folium.Map(location=[40.758896,  -73.985130],tiles='OpenStreetMap',zoom_start=13)

style_function = lambda x: {
    'fillColor': colormap(x['properties']['number_of_trips']),
    'color': 'black',
    'weight':2,
    'fillOpacity':0.5
}

folium.GeoJson(
    manhattan_data,
    name='NYC NTA',
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['NTAName', 'number_of_trips'],
        aliases=['NTA Name', 'Total Trips'], 
        localize=True
    )
).add_to(heatmap)

colormap.add_to(heatmap)
heatmap

In [16]:
mta_stations = gpd.read_file("C:\\Users\\Rodrigo Bernal\\Desktop\\DS4A_workspace\\Datathon\\Dataset\\Dataset\\subway_stations\\geo_export_a1655c7a-e70a-422e-89f7-4a8f915e9c79.shp")
mta_stations.to_file("mta_stations.json", driver="GeoJSON")
mta_stations['lon'] = mta_stations.geometry.x
mta_stations['lat'] = mta_stations.geometry.y

In [62]:
mta_stations

,line,name,notes,objectid,url,geometry,lon,lat
0,4-6-6 Express,Astor Pl,"4 nights, 6-all times, 6 Express-weekdays AM s...",1.0,http://web.mta.info/nyct/service/,POINT (-73.99107 40.73005),-73.991070,40.730054
1,4-6-6 Express,Canal St,"4 nights, 6-all times, 6 Express-weekdays AM s...",2.0,http://web.mta.info/nyct/service/,POINT (-74.00019 40.71880),-74.000193,40.718803
2,1-2,50th St,"1-all times, 2-nights",3.0,http://web.mta.info/nyct/service/,POINT (-73.98385 40.76173),-73.983849,40.761728
3,2-3-4,Bergen St,"4-nights, 3-all other times, 2-all times",4.0,http://web.mta.info/nyct/service/,POINT (-73.97500 40.68086),-73.974999,40.680862
4,3-4,Pennsylvania Ave,"4-nights, 3-all other times",5.0,http://web.mta.info/nyct/service/,POINT (-73.89489 40.66471),-73.894886,40.664714
...,...,...,...,...,...,...,...,...
468,D-F-N-Q,Coney Island - Stillwell Av,"D,F,N,Q-all times",469.0,http://web.mta.info/nyct/service/,POINT (-73.98124 40.57728),-73.981236,40.577281
469,7-7 Express,34th St - Hudson Yards,"7-all times, 7 Express-rush hours AM westbound...",470.0,http://web.mta.info/nyct/service/,POINT (-74.00220 40.75545),-74.002197,40.755446
470,Q,72nd St,Q-all times,641.0,http://web.mta.info/nyct/service/,POINT (-73.95836 40.76880),-73.958362,40.768803
471,Q,86th St,Q-all times,642.0,http://web.mta.info/nyct/service/,POINT (-73.95177 40.77786),-73.951771,40.777861


In [17]:
type(mta_stations)
latitudes = list(mta_stations.lat)
longitudes = list(mta_stations.lon)
names=list(mta_stations.name)
for lat, lng,name in zip(latitudes, longitudes,names):
    folium.Marker(
      location = [lat, lng], 
      popup = name,
      icon = folium.Icon(color='blue', icon='subway',prefix='fa')
     ).add_to(heatmap)

In [18]:
#folium.GeoJson(mta_stations).add_to(heatmap)
#heatmap
#folium.Marker(mta_stations.geometry).add_to(heatmap)
heatmap

In [19]:
heatmap.save("heatmap.png")